### Learning to make a tensorflow dataset

In [20]:
# creating tfrecord file

import tensorflow as tf
import os
from matplotlib.image import imread

# Define the paths to your image folders
folder_paths = {
    'Bhaktapur-Durbar-Square': '/Users/nirajanpaudel17/Documents/Python/Major-Project/Image-Processing/Dataset/Bhaktapur-Durbar-Square',
    'Bouddanath': '/Users/nirajanpaudel17/Documents/Python/Major-Project/Image-Processing/Dataset/Bouddanath',
    'Pashupatinath': '/Users/nirajanpaudel17/Documents/Python/Major-Project/Image-Processing/Dataset/Pashupatinath',
    'Patan-Durbar-Square': '/Users/nirajanpaudel17/Documents/Python/Major-Project/Image-Processing/Dataset/Patan-Durbar-Square',
    'Swyambunath': '/Users/nirajanpaudel17/Documents/Python/Major-Project/Image-Processing/Dataset/Swyambunath'
}

# Define the output TFRecord file name
output_directory = '/Users/nirajanpaudel17/Documents/Python/Major-Project/Temple-Classification'
output_filename = os.path.join(output_directory, 'labeled_temples_images.tfrecord')

# Initialize an empty list to store the labeled images
labeled_images = []

# Iterate over each class folder
for class_name, folder_path in folder_paths.items():
    
    # Get the list of image filenames in the current class folder
    image_filenames = os.listdir(folder_path)
    

    # Iterate over each image in the class folder
    for image_filename in image_filenames:
        # Create the full path to the image file
        image_path = os.path.join(folder_path, image_filename)

        try:
            # Read the image file
            image = tf.io.read_file(image_path)

            # Decode the image file
            image = tf.io.decode_jpeg(image)
            
            image = tf.io.encode_jpeg(image).numpy()


            # Create a labeled example
            example = tf.train.Example(features=tf.train.Features(feature={
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
                'class': tf.train.Feature(bytes_list=tf.train.BytesList(value=[class_name.encode()]))
            }))

            # Append the labeled example to the list
            labeled_images.append(example)

        except tf.errors.InvalidArgumentError:
            print('Skipping unsupported image:', image_path)

# Create a writer for the TFRecord file
with tf.io.TFRecordWriter(output_filename) as writer:
    # Write each labeled example to the TFRecord file
    for example in labeled_images:
        writer.write(example.SerializeToString())

# # Print the number of labeled images
print('Number of labeled images:', len(labeled_images))


Number of labeled images: 1639


In [34]:
import tensorflow as tf


def parse_tfrecord_fn(example_proto):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'class': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example_proto, feature_description)
    
    image = tf.image.decode_jpeg(example['image'], channels=3)  # Modify the decoding function as per image format
    image = tf.cast(image, tf.float32) / 255.0
    
    label = tf.cast(example['class'], tf.string)
    
    return image, label


# Specify the path to the TFRecord file(s)
tfrecord_files = ['/Users/nirajanpaudel17/Documents/Python/Major-Project/Temple-Classification/labeled_temples_images.tfrecord']

# Create a TFRecordDataset
dataset = tf.data.TFRecordDataset(tfrecord_files)

# Apply parsing function to the dataset
dataset = dataset.map(parse_tfrecord_fn)

# Print the first few examples in the dataset
for image, label in dataset.take(1):
    # Process or use the image and label as needed
    print(image.shape, label)


(300, 300, 3) tf.Tensor(b'Bhaktapur-Durbar-Square', shape=(), dtype=string)


In [37]:
# splitting the datset into train and test set

dataset = dataset.shuffle(buffer_size=1000, seed=42)

# Split the dataset into train and test sets

num_samples = int(dataset.cardinality().numpy())

train_size = int(0.8 * num_samples)  # 80% for training
test_size = dataset.cardinality() - train_size

train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

# Print the number of samples in each set
print("Train set size:", train_size)
print("Test set size:", test_size)

Train set size: -1
Test set size: tf.Tensor(-1, shape=(), dtype=int64)


In [40]:
for image, label in train_dataset.take(3):
    # Process or use the image and label as needed
    print(image.shape, label)

(300, 300, 3) tf.Tensor(b'Bouddanath', shape=(), dtype=string)
(300, 300, 3) tf.Tensor(b'Bouddanath', shape=(), dtype=string)
(300, 300, 3) tf.Tensor(b'Pashupatinath', shape=(), dtype=string)
